<a href="https://colab.research.google.com/github/allen44/riiid-test-answer-prediction/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%cd /content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/
%pwd

/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction


'/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction'

In [39]:
% pip install featuretools dask distributed tornado
import featuretools as ft

     |████████████████████████████████| 296kB 5.1MB/s 
     |████████████████████████████████| 675kB 7.1MB/s 
ERROR: featuretools 0.23.2 has requirement dask[dataframe]>=2.12.0, but you'll have dask 0.19.4 which is incompatible.
ERROR: featuretools 0.23.2 has requirement distributed>=2.12.0, but you'll have distributed 1.25.3 which is incompatible.
  Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0


/usr/local/lib/python3.7/dist-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load data from csv

Using our insights gained from the EDA, when can import the data from csv with an get right on to feature engineering.

In [2]:
#Choose pickle, or csv
suffix = '.pkl.gzip'
# suffix = '.csv'

pwd = '/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction'

# #Define data paths
lectures_path = f'{pwd}/data/raw/lectures{suffix}'
questions_path = f'{pwd}/data/raw/questions{suffix}'
train_path = f'{pwd}/data/raw/train{suffix}'

lectures_path, questions_path, train_path

('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/raw/lectures.pkl.gzip',
 '/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/raw/questions.pkl.gzip',
 '/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/raw/train.pkl.gzip')

In [3]:
if suffix == '.csv':
  df_lectures = pd.read_csv(lectures_path)
  df_questions = pd.read_csv(questions_path)
  df_train = pd.read_csv(train_path)
elif suffix == '.pkl.gzip':
  df_lectures = pd.read_pickle(lectures_path)
  df_questions = pd.read_pickle(questions_path)
  df_train = pd.read_pickle(train_path)


df_lectures.shape, df_questions.shape, df_train.shape

((418, 4), (13523, 5), (101230332, 10))

In [4]:
# # Save to piclkle for faster loading later
# df_lectures.to_pickle(lectures_path)
# df_questions.to_pickle(questions_path)
# df_train.to_pickle(train_path)


In [5]:
df_lectures.shape, df_questions.shape, df_train.shape

((418, 4), (13523, 5), (101230332, 10))

## # Use a subset of users in df_train

Feature engineering can be computationally expensive, so we'll use a subset for this analysis

In [6]:
# Check number of total users
unique_users = df_train['user_id'].unique()
len(unique_users)

393656

In [10]:
# Check size of the train df for this user subset
# unique_users_subset = df_train[df_train['user_id'] % 10000 == 0]['user_id']
unique_users_subset = df_train[df_train['user_id']==801103753]['user_id']
len(unique_users_subset)

17917

In [11]:
# Check number of total users in this subset
len(unique_users_subset.unique())

1

In [12]:
unique_users_subset.index

Int64Index([37497771, 37497772, 37497773, 37497774, 37497775, 37497776,
            37497777, 37497778, 37497779, 37497780,
            ...
            37515678, 37515679, 37515680, 37515681, 37515682, 37515683,
            37515684, 37515685, 37515686, 37515687],
           dtype='int64', length=17917)

In [13]:
# Make the subset
df_train = df_train[df_train.index.isin(unique_users_subset.index)]

df_train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
37497771,37497771,0,801103753,4886,0,0,1,0,NaN,NaN
37497772,37497772,21186,801103753,4032,0,1,3,0,249000.0,False
37497773,37497773,55030,801103753,5927,0,2,1,1,18000.0,False
37497774,37497774,81839,801103753,4754,0,3,2,1,31000.0,False
37497775,37497775,103346,801103753,6336,0,4,1,0,24000.0,False
...,...,...,...,...,...,...,...,...,...,...
37515683,37515683,28340635545,801103753,13507,0,9995,0,1,36000.0,True
37515684,37515684,28340706852,801103753,9203,0,9996,2,1,136000.0,True
37515685,37515685,28340784904,801103753,9819,0,9997,0,0,35000.0,True
37515686,37515686,28340853513,801103753,9506,0,9999,1,1,44000.0,True


In [14]:
# Make the timestamps unique by adding judder to any non-unique timestamps
while not df_train['timestamp'].is_unique:
  for idx in list(df_train[df_train['timestamp'].duplicated()].index):
   df_train.loc[idx, 'timestamp'] = df_train.loc[idx, 'timestamp'] + np.random.randint(0, 10)

#Check for any remaining
df_train[df_train['timestamp'].duplicated()].index

Int64Index([], dtype='int64')

## Set the dtypes

In [15]:
# Check the dtypes before
print(df_lectures.dtypes, '\n')
print(df_questions.dtypes, '\n')
print(df_train.dtypes, '\n')

lecture_id     int64
tag            int64
part           int64
type_of       object
dtype: object 

question_id        int64
bundle_id          int64
correct_answer     int64
part               int64
tags              object
dtype: object 

row_id                              int64
timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object 



In [16]:
from datetime import datetime

# Set the dtype of each column, 

#Lectures
for col in df_lectures.columns:
  df_lectures[col] = pd.Categorical(df_lectures[col])

# Questions
for col in df_questions.drop(columns='tags').columns:
  df_questions[col] = pd.Categorical(df_questions[col])

# Train_questions: convert categorical columns
categorical_columns = ['user_id', 
                       'content_id', 
                       'content_type_id', 
                       'task_container_id', 
                       'user_answer', 
                       'answered_correctly',
                       'prior_question_had_explanation']

for column_id in categorical_columns:
  df_train[column_id] = pd.Categorical(df_train[column_id])

# Convert timestamp in milliseconds to datetime
from datetime import datetime

if df_train['timestamp'].dtype == 'int':
  max_timestamp = df_train['timestamp'].max()
  diff = datetime(2020, 10, 10) - pd.to_datetime(max_timestamp * 10**6)
  # df_train['timestamp_int'] = df_train['timestamp']
  df_train['timestamp'] = diff + pd.to_datetime(df_train['timestamp'] * 10**6)

# Convert prior_question_elapsed_time to timedelta
df_train['prior_question_elapsed_time'] = pd.to_timedelta(df_train['prior_question_elapsed_time'])
  
# Split lectures watched and questions answered into seperate data frames

# Handle lecture_id, content_id, and question_id
# Get and set the union of the three cat dtypes
categories = set(df_train['content_id'].dtype.categories)\
            .union(set(df_lectures['lecture_id'].dtype.categories)\
            .union(set(df_questions['question_id'].dtype.categories)))

df_train['content_id'] = pd.Categorical(df_train['content_id'], categories=categories)
df_lectures['lecture_id'] = pd.Categorical(df_lectures['lecture_id'], categories=categories)
df_questions['question_id'] = pd.Categorical(df_questions['question_id'], categories=categories)

# Confirm that the dtypes are the same
assert df_train['content_id'].dtype == df_lectures['lecture_id'].dtype
assert df_questions['question_id'].dtype == df_lectures['lecture_id'].dtype

#Split lectures watch and questions_answered into seperate dataframes
df_train_l = pd.DataFrame(df_train[df_train['content_type_id']==1])
df_train_q = pd.DataFrame(df_train[df_train['content_type_id']==0])
# del df_train

#Drop irrelevant columns from new dataframes
df_train_l.drop(columns=['content_type_id', 
                         'user_answer', 
                         'answered_correctly', 
                         'prior_question_elapsed_time', 
                         'prior_question_had_explanation'],
                inplace=True)
df_train_q.drop(columns=['content_type_id'],
                inplace=True)

# Drop remiaining nan rows
df_questions.dropna(inplace=True)
df_train_q.dropna(inplace=True)

# Recategorize the train questions after spliting the dataframes
cat_cols = ['task_container_id', 
            'user_answer']
bool_cols = ['answered_correctly', 
             'prior_question_had_explanation']

for column_id in cat_cols:
  df_train_q[column_id] = pd.Categorical(df_train_q[column_id])
for column_id in bool_cols:
  df_train_q[column_id] = df_train_q[column_id].astype(bool)
  
# # Set a multiindex for df_train_l and df_train_q
# df_train_l.set_index(['user_id', 'timestamp'], inplace=True)
# df_train_q.set_index(['user_id', 'timestamp'], inplace=True)

# # drop the irrelevant row_id
# if 'row_id' in list(df_train_l.columns):
#   df_train_l.drop(columns='row_id', inplace=True)
# if 'row_id' in list(df_train_q.columns):
#   df_train_q.drop(columns='row_id', inplace=True)


# make a cutoff_times dataframe for lectures and questions
df_cutoff_times_l = pd.DataFrame(df_train_l[['user_id', 'timestamp']])
df_cutoff_times_q = pd.DataFrame(df_train_q[['user_id', 'timestamp']])

# Assert that all dataframes have no na values
assert df_lectures.isna().sum().sum()==0
assert df_questions.isna().sum().sum()==0
assert df_train_l.isna().sum().sum()==0
assert df_train_q.isna().sum().sum()==0
assert df_cutoff_times_l.isna().sum().sum()==0
assert df_cutoff_times_q.isna().sum().sum()==0

# Make a users dataframe
df_users = pd.DataFrame(df_train['user_id'].unique(), columns=['user_id'])

In [17]:
# Check the dtypes after
print(df_lectures.dtypes, '\n')
print(df_questions.dtypes, '\n')
print(df_train_l.dtypes, '\n')
print(df_train_q.dtypes, '\n')

lecture_id    category
tag           category
part          category
type_of       category
dtype: object 

question_id       category
bundle_id         category
correct_answer    category
part              category
tags                object
dtype: object 

row_id                        int64
timestamp            datetime64[ns]
user_id                    category
content_id                 category
task_container_id          category
dtype: object 

row_id                                      int64
timestamp                          datetime64[ns]
user_id                                  category
content_id                               category
task_container_id                        category
user_answer                              category
answered_correctly                           bool
prior_question_elapsed_time       timedelta64[ns]
prior_question_had_explanation               bool
dtype: object 



### Binarize the tags column

In [18]:
# split tags
df_questions['tags'] = df_questions['tags'].str.split()

# Binarize the tags
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

labels_df = pd.DataFrame(mlb.fit_transform(df_questions['tags']),
                            columns=mlb.classes_, 
                            index=df_questions.index)

# Drop old column
df_questions.drop(columns=['tags'], inplace=True)

#Rename new columns and set dtype
for label in labels_df.columns:
  new_col = 'tag_'+ label
  df_questions[new_col] = labels_df[label].astype(bool)

df_questions

,question_id,bundle_id,correct_answer,part,tag_0,tag_1,tag_10,tag_100,tag_101,tag_102,tag_103,tag_104,tag_105,tag_106,tag_107,tag_108,tag_109,tag_11,tag_110,tag_111,tag_112,tag_113,tag_114,tag_115,tag_116,tag_117,tag_118,tag_119,tag_12,tag_120,tag_121,tag_122,tag_123,tag_124,tag_125,tag_126,tag_127,tag_128,tag_129,tag_13,...,tag_63,tag_64,tag_65,tag_66,tag_67,tag_68,tag_69,tag_7,tag_70,tag_71,tag_72,tag_73,tag_74,tag_75,tag_76,tag_77,tag_78,tag_79,tag_8,tag_80,tag_81,tag_82,tag_83,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_9,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,1,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,2,0,1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,3,3,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,4,4,3,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13518,13518,13518,3,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
13519,13519,13519,3,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,.

# Prepare data


## Create Entity Set

In [40]:
# Define empty entity set
es = ft.EntitySet('riid')

In [41]:
# Add df_lectures to entity set
es.entity_from_dataframe(entity_id='lectures', 
                         dataframe=df_lectures, 
                         index='lecture_id')

Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
  Relationships:
    No relationships

In [42]:
# Add df_questions to entity set
es.entity_from_dataframe(entity_id='questions', 
                         dataframe=df_questions, 
                         index='question_id')

Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
  Relationships:
    No relationships

In [43]:
# Add df_train_l to entity set
es.entity_from_dataframe(entity_id='train_l', 
                         dataframe=df_train_l,
                         index='row_id',
                         time_index='timestamp',
                        )

Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
    train_l [Rows: 308, Columns: 5]
  Relationships:
    No relationships

In [44]:
variable_types = {
    'prior_question_elapsed_time': 'timedelta'
}

# Add df_train_q to entity set
es.entity_from_dataframe(entity_id='train_q', 
                         dataframe=df_train_q,
                         index='row_id',
                         variable_types=variable_types,
                         time_index='timestamp',
                         )

Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
    train_l [Rows: 308, Columns: 5]
    train_q [Rows: 17608, Columns: 9]
  Relationships:
    No relationships

In [45]:
# Add df_users to entity set
es.entity_from_dataframe(entity_id='users', 
                         dataframe=df_users,
                         index='user_id',
                        #  make_index = False,
                        #  time_index ='timestamp',
                         )

Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
    train_l [Rows: 308, Columns: 5]
    train_q [Rows: 17608, Columns: 9]
    users [Rows: 1, Columns: 1]
  Relationships:
    No relationships

In [46]:
r_lectures_train_l = ft.Relationship(es["lectures"]["lecture_id"],
                                          es["train_l"]["content_id"])

r_questions_train_q = ft.Relationship(es["questions"]["question_id"],
                                    es["train_q"]["content_id"])

r_users_train_l = ft.Relationship(es["users"]["user_id"],
                                    es["train_l"]["user_id"])

r_users_train_q = ft.Relationship(es["users"]["user_id"],
                                    es["train_q"]["user_id"])


es = es.add_relationship(r_lectures_train_l)
es = es.add_relationship(r_questions_train_q)
es = es.add_relationship(r_users_train_l)
es = es.add_relationship(r_users_train_q)
es = es.add_relationship(r_users_train_q)
es

/usr/local/lib/python3.7/dist-packages/featuretools/entityset/entityset.py:251: UserWarning: Not adding duplicate relationship: <Relationship: train_q.user_id -> users.user_id>
  "Not adding duplicate relationship: " + str(relationship))


Entityset: riid
  Entities:
    lectures [Rows: 418, Columns: 4]
    questions [Rows: 13522, Columns: 192]
    train_l [Rows: 308, Columns: 5]
    train_q [Rows: 17608, Columns: 9]
    users [Rows: 1, Columns: 1]
  Relationships:
    train_l.content_id -> lectures.lecture_id
    train_q.content_id -> questions.question_id
    train_l.user_id -> users.user_id
    train_q.user_id -> users.user_id

## Check the baseline primitive set

In [47]:
# Initialize empty dictionaries
feature_defs = {}
feature_matrix = {}
where_primitives = {}
agg_primitives = {}
trans_primitives = {}

In [48]:
#Set interesting values
es['train_q']['answered_correctly'].interesting_values = [0, 1]

### Check primitives for lectures

In [49]:
entity = 'lectures'

where_primitives[entity] = ['sum', 'mean']
agg_primitives[entity] = []
trans_primitives[entity] = []

feature_defs[entity] = ft.dfs(entityset=es, 
                      target_entity=entity, 
                      where_primitives = where_primitives[entity],
                      max_depth=3, 
                      features_only=True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.\n')

This will generate 80 features for lectures.



/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['mean', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [50]:
print(entity, '\n')
feature_defs[entity]

lectures 



[<Feature: tag>,
 <Feature: part>,
 <Feature: type_of>,
 <Feature: COUNT(train_l)>,
 <Feature: MODE(train_l.task_container_id)>,
 <Feature: MODE(train_l.user_id)>,
 <Feature: NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: NUM_UNIQUE(train_l.user_id)>,
 <Feature: MODE(train_l.DAY(timestamp))>,
 <Feature: MODE(train_l.MONTH(timestamp))>,
 <Feature: MODE(train_l.WEEKDAY(timestamp))>,
 <Feature: MODE(train_l.YEAR(timestamp))>,
 <Feature: NUM_UNIQUE(train_l.DAY(timestamp))>,
 <Feature: NUM_UNIQUE(train_l.MONTH(timestamp))>,
 <Feature: NUM_UNIQUE(train_l.WEEKDAY(timestamp))>,
 <Feature: NUM_UNIQUE(train_l.YEAR(timestamp))>,
 <Feature: MAX(train_l.users.COUNT(train_l))>,
 <Feature: MAX(train_l.users.COUNT(train_q))>,
 <Feature: MAX(train_l.users.NUM_UNIQUE(train_l.content_id))>,
 <Feature: MAX(train_l.users.NUM_UNIQUE(train_l.task_container_id))>,
 <Feature: MAX(train_l.users.NUM_UNIQUE(train_q.content_id))>,
 <Feature: MAX(train_l.users.NUM_UNIQUE(train_q.task_container_id))>,
 <Feature:

### Check primitives for questions

In [51]:
entity = 'questions'

# where_primitives[entity] = ['sum', 'mean']
# agg_primitives[entity] = []
# trans_primitives[entity] = []


feature_defs[entity] = ft.dfs(entityset=es, 
                      target_entity='questions', 
                      where_primitives = ['sum', 'mean'],
                      max_depth=3, 
                      features_only=True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.\n')

This will generate 308 features for questions.



In [52]:
print(entity, '\n')
feature_defs[entity]

questions 



[<Feature: bundle_id>,
 <Feature: correct_answer>,
 <Feature: part>,
 <Feature: tag_0>,
 <Feature: tag_1>,
 <Feature: tag_10>,
 <Feature: tag_100>,
 <Feature: tag_101>,
 <Feature: tag_102>,
 <Feature: tag_103>,
 <Feature: tag_104>,
 <Feature: tag_105>,
 <Feature: tag_106>,
 <Feature: tag_107>,
 <Feature: tag_108>,
 <Feature: tag_109>,
 <Feature: tag_11>,
 <Feature: tag_110>,
 <Feature: tag_111>,
 <Feature: tag_112>,
 <Feature: tag_113>,
 <Feature: tag_114>,
 <Feature: tag_115>,
 <Feature: tag_116>,
 <Feature: tag_117>,
 <Feature: tag_118>,
 <Feature: tag_119>,
 <Feature: tag_12>,
 <Feature: tag_120>,
 <Feature: tag_121>,
 <Feature: tag_122>,
 <Feature: tag_123>,
 <Feature: tag_124>,
 <Feature: tag_125>,
 <Feature: tag_126>,
 <Feature: tag_127>,
 <Feature: tag_128>,
 <Feature: tag_129>,
 <Feature: tag_13>,
 <Feature: tag_130>,
 <Feature: tag_131>,
 <Feature: tag_132>,
 <Feature: tag_133>,
 <Feature: tag_134>,
 <Feature: tag_135>,
 <Feature: tag_136>,
 <Feature: tag_137>,
 <Feature: tag_

### Check primitives for train_l

In [53]:
entity = 'train_l'

# where_primitives[entity] = ['sum', 'mean']
# agg_primitives[entity] = []
# trans_primitives[entity] = []


feature_defs[entity] = ft.dfs(entityset=es, 
                      target_entity=entity, 
                      where_primitives = ['sum', 'mean'],
                      max_depth=3, 
                      features_only=True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.\n')

This will generate 247 features for train_l.



/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['mean', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [54]:
print(entity, '\n')
feature_defs[entity]

train_l 



[<Feature: user_id>,
 <Feature: content_id>,
 <Feature: task_container_id>,
 <Feature: DAY(timestamp)>,
 <Feature: MONTH(timestamp)>,
 <Feature: WEEKDAY(timestamp)>,
 <Feature: YEAR(timestamp)>,
 <Feature: lectures.tag>,
 <Feature: lectures.part>,
 <Feature: lectures.type_of>,
 <Feature: lectures.COUNT(train_l)>,
 <Feature: lectures.MODE(train_l.task_container_id)>,
 <Feature: lectures.MODE(train_l.user_id)>,
 <Feature: lectures.NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: lectures.NUM_UNIQUE(train_l.user_id)>,
 <Feature: users.COUNT(train_l)>,
 <Feature: users.MODE(train_l.content_id)>,
 <Feature: users.MODE(train_l.task_container_id)>,
 <Feature: users.NUM_UNIQUE(train_l.content_id)>,
 <Feature: users.NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: users.COUNT(train_q)>,
 <Feature: users.MODE(train_q.content_id)>,
 <Feature: users.MODE(train_q.task_container_id)>,
 <Feature: users.MODE(train_q.user_answer)>,
 <Feature: users.NUM_UNIQUE(train_q.content_id)>,
 <Feature: users.

### Check primitives for train_q

In [55]:
entity = 'train_q'

# where_primitives[entity] = ['sum', 'mean']
# agg_primitives[entity] = []
# trans_primitives[entity] = []


feature_defs[entity] = ft.dfs(entityset=es, 
                      target_entity=entity, 
                      where_primitives = ['sum', 'mean'],
                      max_depth=3, 
                      features_only=True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.\n')

This will generate 254 features for train_q.



/usr/local/lib/python3.7/dist-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  where_primitives: ['mean', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [56]:
print(entity, '\n')
feature_defs[entity]

train_q 



[<Feature: user_id>,
 <Feature: content_id>,
 <Feature: task_container_id>,
 <Feature: user_answer>,
 <Feature: answered_correctly>,
 <Feature: prior_question_had_explanation>,
 <Feature: DAY(timestamp)>,
 <Feature: MONTH(timestamp)>,
 <Feature: WEEKDAY(timestamp)>,
 <Feature: YEAR(timestamp)>,
 <Feature: questions.bundle_id>,
 <Feature: questions.correct_answer>,
 <Feature: questions.part>,
 <Feature: questions.tag_0>,
 <Feature: questions.tag_1>,
 <Feature: questions.tag_10>,
 <Feature: questions.tag_100>,
 <Feature: questions.tag_101>,
 <Feature: questions.tag_102>,
 <Feature: questions.tag_103>,
 <Feature: questions.tag_104>,
 <Feature: questions.tag_105>,
 <Feature: questions.tag_106>,
 <Feature: questions.tag_107>,
 <Feature: questions.tag_108>,
 <Feature: questions.tag_109>,
 <Feature: questions.tag_11>,
 <Feature: questions.tag_110>,
 <Feature: questions.tag_111>,
 <Feature: questions.tag_112>,
 <Feature: questions.tag_113>,
 <Feature: questions.tag_114>,
 <Feature: questions.t

### Check primitives for users

In [57]:
entity = 'users'

# where_primitives[entity] = ['sum', 'mean']
# agg_primitives[entity] = []
# trans_primitives[entity] = []


feature_defs[entity] = ft.dfs(entityset=es, 
                      target_entity=entity, 
                      where_primitives = ['sum', 'mean'],
                      max_depth=3, 
                      features_only=True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.\n')

This will generate 318 features for users.



In [58]:
print(entity, '\n')
feature_defs[entity]

users 



[<Feature: COUNT(train_l)>,
 <Feature: MODE(train_l.content_id)>,
 <Feature: MODE(train_l.task_container_id)>,
 <Feature: NUM_UNIQUE(train_l.content_id)>,
 <Feature: NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: COUNT(train_q)>,
 <Feature: MODE(train_q.content_id)>,
 <Feature: MODE(train_q.task_container_id)>,
 <Feature: MODE(train_q.user_answer)>,
 <Feature: NUM_UNIQUE(train_q.content_id)>,
 <Feature: NUM_UNIQUE(train_q.task_container_id)>,
 <Feature: NUM_UNIQUE(train_q.user_answer)>,
 <Feature: PERCENT_TRUE(train_q.answered_correctly)>,
 <Feature: PERCENT_TRUE(train_q.prior_question_had_explanation)>,
 <Feature: MODE(train_l.DAY(timestamp))>,
 <Feature: MODE(train_l.MONTH(timestamp))>,
 <Feature: MODE(train_l.WEEKDAY(timestamp))>,
 <Feature: MODE(train_l.YEAR(timestamp))>,
 <Feature: MODE(train_l.lectures.part)>,
 <Feature: MODE(train_l.lectures.tag)>,
 <Feature: MODE(train_l.lectures.type_of)>,
 <Feature: NUM_UNIQUE(train_l.DAY(timestamp))>,
 <Feature: NUM_UNIQUE(train_l.MONTH(

## Define the complete primitive set

### Aggregation Primitives

In [59]:
all_p = ft.list_primitives()
trans_p = all_p.loc[all_p['type'] == 'transform'].copy()
agg_p = all_p.loc[all_p['type'] == 'aggregation'].copy()

pd.options.display.max_colwidth = 100
# agg_p

In [60]:
# Specify aggregation primitives
agg_primitives = ['sum', 'time_since_last', 'avg_time_between', 'all', 'mode', 'num_unique', 'min', 'last', 
                  'mean', 'percent_true', 'max', 'std', 'count']

### Transform Primitives

In [61]:
# trans_p

In [62]:
# Specify transformation primitives
trans_primitives = ['cum_sum', 'diff', 'time_since_previous']

### Where Primitives

These primitives are applied to the `interesting_values` to build conditional features. 

In [63]:
# Specify where primitives
where_primitives = ['sum', 'mean', 'percent_true', 'all', 'any']

### Custom Primitives

For this problem, I wrote a custom primitive that calculates the sum of a value in the month prior to the cutoff time.

The second custom primitive finds the time since a previous true value. It simply finds the time between True examples.

In [64]:
from datetime import datetime

def total_previous_month(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_one_month_earlier = time - pd.DateOffset(months=1)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_one_month_earlier)]
    total = df['value'].sum()
    
    return total


def total_previous_week(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_one_week_earlier = time - pd.DateOffset(weeks=1)
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_one_week_earlier)]
    total = df['value'].sum()
 
    return total

def total_previous_day(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_one_day_earlier = time - pd.DateOffset(days=1)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_one_day_earlier)]
    total = df['value'].sum()
    
    return total


def total_previous_half_day(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_12_hours_earlier = time - pd.DateOffset(hours=12)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_12_hours_earlier)]
    total = df['value'].sum()
    
    return total

def total_previous_quarter_day(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_6_hours_earlier = time - pd.DateOffset(hours=6)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_6_hours_earlier)]
    total = df['value'].sum()
    
    return total


def total_previous_3_hours(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_3_hours_earlier = time - pd.DateOffset(hours=3)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_3_hours_earlier)]
    total = df['value'].sum()
    
    return total


def total_previous_2_hours(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_2_hours_earlier = time - pd.DateOffset(hours=2)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_2_hours_earlier)]
    total = df['value'].sum()
    
    return total

def total_previous_1_hour(numeric, datetime, time):
    """Return total of `numeric` column in the month prior to `time`."""
    df = pd.DataFrame({'value': numeric, 'date': datetime})
    time_1_hour_earlier = time - pd.DateOffset(hours=1)
   
        
    # Filter data and sum up total
    df = df[(df['date'] < time) & (df['date'] >= time_1_hour_earlier)]
    total = df['value'].sum()

    return total

def time_since_true(boolean, datetime):
    """Calculate time since previous true value"""
    
    if np.any(np.array(list(boolean)) == 1):
        # Create dataframe sorted from oldest to newest 
        df = pd.DataFrame({'value': boolean, 'date': datetime}).\
                sort_values('date', ascending = False).reset_index()

        older_date = None

        # Iterate through each date in reverse order
        for date in df.loc[df['value'] == 1, 'date']:

            # If there was no older true value
            if older_date == None:
                # Subset to times on or after true
                times_after_idx = df.loc[df['date'] >= date].index

            else:
                # Subset to times on or after true but before previous true
                times_after_idx = df.loc[(df['date'] >= date) & (df['date'] < older_date)].index
            older_date = date
            # Calculate time since previous true
            df.loc[times_after_idx, 'time_since_previous'] = (df.loc[times_after_idx, 'date'] - date).dt.total_seconds()

        return list(df['time_since_previous'])[::-1]
    
    # Handle case with no true values
    else:
        return [np.nan for _ in range(len(boolean))]

#### Custom Primitive Implementation

Making a custom primitive is simple: first we define a function (`total_previous_month`) and then we `make_agg_primitive` with `input_type[s]`, a `return_type`, and whether or not the primitive requires the `cutoff_time` through `uses_calc_time`. 

This primitive is an aggregation primitive because it takes in multiple numbers - transactions for the previous month - and returns a single number - the total of the transactions. 

In [65]:
from featuretools.primitives import make_agg_primitive

# Takes in a number and outputs a number
total_previous_m = make_agg_primitive(total_previous_month, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_w = make_agg_primitive(total_previous_week, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_d = make_agg_primitive(total_previous_day, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_12h = make_agg_primitive(total_previous_half_day, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_6h = make_agg_primitive(total_previous_quarter_day, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_3h = make_agg_primitive(total_previous_3_hours, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_2h = make_agg_primitive(total_previous_2_hours, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)
total_previous_1h = make_agg_primitive(total_previous_1_hour, 
                                    input_types = [ft.variable_types.Numeric,
                                                   ft.variable_types.Datetime],
                                    return_type = ft.variable_types.Numeric, 
                                    uses_calc_time = True)

In [66]:
from featuretools.primitives import make_trans_primitive

# # Specify the inputs and return
# time_since = make_trans_primitive(time_since_true, 
#                                   input_types = [ft.variable_types.Boolean, 
#                                                   ft.variable_types.Datetime],
#                                   return_type = ft.variable_types.Numeric)

Now just have to pass this in as another aggregation primitive for Featuretools to use it in calculations.



Let's add the two custom primitives to the respective lists. In the final version of feature engineering, I did not use the `time_since` primitive. I ran into problems with the implementation but would encourage anyone to try and fix it or build their own custom primitive[s].

In [67]:
agg_primitives.append(total_previous_m)
agg_primitives.append(total_previous_w)
agg_primitives.append(total_previous_d)
agg_primitives.append(total_previous_12h)
agg_primitives.append(total_previous_6h)
agg_primitives.append(total_previous_3h)
agg_primitives.append(total_previous_2h)
agg_primitives.append(total_previous_1h)

# trans_primitives.append(time_since)

# Deep Feature Synthesis with Specified Primitives

We'll again run Deep Feature Synthesis to make the feature definitions this time using the selected primitives and the custom primitives. 

In [79]:
# Initialize empty dictionaries
entities = set()

feature_defs = {}
feature_matrix = {}
agg_primitives = {}
trans_primitives= {}
where_primitives ={}
cutoff_time = {}
time_index = {}
max_depth = {}

## lectures

In [80]:
# Lectures

entity = 'lectures'
entities.add(entity)

agg_primitives[entity] = [
                          # 'sum', 
                          'time_since_last', 
                          'avg_time_between', 
                          # 'all', 
                          'mode', 
                          'num_unique', 
                          # 'min', 
                          'last', 
                          # 'mean', 
                          # 'percent_true', 
                          # 'max', 
                          # 'std', 
                          'count',
                          # total_previous_m,
                          # total_previous_w,
                          # total_previous_d,
                          # total_previous_12h,
                          # total_previous_6h,
                          # total_previous_3h,
                          # total_previous_2h,
                          # total_previous_1h
                          ]

trans_primitives[entity] = [
                          # 'cum_sum', 
                          # 'diff', 
                          # 'time_since_previous',
                          # time_since
                          ]

where_primitives[entity] = [
                            # 'sum', 
                            # 'mean', 
                            # 'percent_true', 
                            # 'all', 
                            # 'any'
                            ]
max_depth[entity] = 2


feature_defs[entity] = ft.dfs(entityset=es, 
                              target_entity=entity, 
                              # cutoff_time = cutoff_times, 
                              agg_primitives = agg_primitives[entity],
                              trans_primitives = trans_primitives[entity],
                              where_primitives = where_primitives[entity],
                              chunk_size = 1000, #len(cutoff_times), 
                              # cutoff_time_in_index = True,
                              max_depth = max_depth[entity],
                              features_only = True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.')

This will generate 13 features for lectures.


In [81]:
print(entity, '\n')
feature_defs[entity]

lectures 



[<Feature: tag>,
 <Feature: part>,
 <Feature: type_of>,
 <Feature: AVG_TIME_BETWEEN(train_l.timestamp)>,
 <Feature: COUNT(train_l)>,
 <Feature: LAST(train_l.row_id)>,
 <Feature: LAST(train_l.task_container_id)>,
 <Feature: LAST(train_l.user_id)>,
 <Feature: MODE(train_l.task_container_id)>,
 <Feature: MODE(train_l.user_id)>,
 <Feature: NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: NUM_UNIQUE(train_l.user_id)>,
 <Feature: TIME_SINCE_LAST(train_l.timestamp)>]

## questions

In [82]:
# Questions
entity = 'questions'
entities.add(entity)

agg_primitives[entity] = [
                          # 'sum', 
                          'time_since_last', 
                          'avg_time_between', 
                          'all', 
                          'mode', 
                          'num_unique', 
                          # 'min', 
                          'last', 
                          # 'mean', 
                          'percent_true', 
                          # 'max', 
                          # 'std', 
                          'count',
                          # total_previous_m,
                          # total_previous_w,
                          # total_previous_d,
                          # total_previous_12h,
                          # total_previous_6h,
                          # total_previous_3h,
                          # total_previous_2h,
                          # total_previous_1h
                          ]

trans_primitives[entity] = [
                          # 'cum_sum', 
                          # 'diff', 
                          # 'time_since_previous',
                          # time_since
                          ]

where_primitives[entity] = [
                            # 'sum', 
                            # 'mean', 
                            'percent_true', 
                            'all', 
                            # 'any'
                            ]
max_depth[entity] = 2


feature_defs[entity] = ft.dfs(entityset=es, target_entity=entity, 
                      # cutoff_time = cutoff_times, 
                      agg_primitives = agg_primitives[entity],
                      trans_primitives = trans_primitives[entity],
                      where_primitives = where_primitives[entity],
                      chunk_size = 1000, #len(cutoff_times), 
                      # cutoff_time_in_index = True,
                      max_depth = max_depth[entity],
                      features_only = True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.')

This will generate 214 features for questions.


In [83]:
print(entity, '\n')
feature_defs[entity]

questions 



[<Feature: bundle_id>,
 <Feature: correct_answer>,
 <Feature: part>,
 <Feature: tag_0>,
 <Feature: tag_1>,
 <Feature: tag_10>,
 <Feature: tag_100>,
 <Feature: tag_101>,
 <Feature: tag_102>,
 <Feature: tag_103>,
 <Feature: tag_104>,
 <Feature: tag_105>,
 <Feature: tag_106>,
 <Feature: tag_107>,
 <Feature: tag_108>,
 <Feature: tag_109>,
 <Feature: tag_11>,
 <Feature: tag_110>,
 <Feature: tag_111>,
 <Feature: tag_112>,
 <Feature: tag_113>,
 <Feature: tag_114>,
 <Feature: tag_115>,
 <Feature: tag_116>,
 <Feature: tag_117>,
 <Feature: tag_118>,
 <Feature: tag_119>,
 <Feature: tag_12>,
 <Feature: tag_120>,
 <Feature: tag_121>,
 <Feature: tag_122>,
 <Feature: tag_123>,
 <Feature: tag_124>,
 <Feature: tag_125>,
 <Feature: tag_126>,
 <Feature: tag_127>,
 <Feature: tag_128>,
 <Feature: tag_129>,
 <Feature: tag_13>,
 <Feature: tag_130>,
 <Feature: tag_131>,
 <Feature: tag_132>,
 <Feature: tag_133>,
 <Feature: tag_134>,
 <Feature: tag_135>,
 <Feature: tag_136>,
 <Feature: tag_137>,
 <Feature: tag_

## train_l

In [84]:
# Train
entity = 'train_l'
entities.add(entity)

agg_primitives[entity] = [
                          # 'sum', 
                          'time_since_last', 
                          'avg_time_between', 
                          'all', 
                          'mode', 
                          'num_unique', 
                          # 'min', 
                          'last', 
                          # 'mean', 
                          'percent_true', 
                          # 'max', 
                          # 'std', 
                          'count',
                          # total_previous_m,
                          # total_previous_w,
                          # total_previous_d,
                          # total_previous_12h,
                          # total_previous_6h,
                          # total_previous_3h,
                          # total_previous_2h,
                          # total_previous_1h
                          ]

trans_primitives[entity] = [
                          # 'cum_sum', 
                          # 'diff', 
                          'time_since_previous',
                          # time_since
                          ]

where_primitives[entity] = [
                            # 'sum', 
                            # 'mean', 
                            'percent_true', 
                            'all', 
                            # 'any'
                            ]
max_depth[entity] = 2


feature_defs[entity] = ft.dfs(entityset=es, target_entity=entity, 
                      # cutoff_time = cutoff_times, 
                      agg_primitives = agg_primitives[entity],
                      trans_primitives = trans_primitives[entity],
                      where_primitives = where_primitives[entity],
                      chunk_size = 1000, #len(cutoff_times), 
                      # cutoff_time_in_index = True,
                      max_depth = max_depth[entity], 
                      features_only = True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.')


This will generate 50 features for train_l.


In [85]:
print(entity, '\n')
feature_defs[entity]

train_l 



[<Feature: user_id>,
 <Feature: content_id>,
 <Feature: task_container_id>,
 <Feature: TIME_SINCE_PREVIOUS(timestamp)>,
 <Feature: lectures.tag>,
 <Feature: lectures.part>,
 <Feature: lectures.type_of>,
 <Feature: lectures.AVG_TIME_BETWEEN(train_l.timestamp)>,
 <Feature: lectures.COUNT(train_l)>,
 <Feature: lectures.LAST(train_l.row_id)>,
 <Feature: lectures.LAST(train_l.task_container_id)>,
 <Feature: lectures.LAST(train_l.user_id)>,
 <Feature: lectures.MODE(train_l.task_container_id)>,
 <Feature: lectures.MODE(train_l.user_id)>,
 <Feature: lectures.NUM_UNIQUE(train_l.task_container_id)>,
 <Feature: lectures.NUM_UNIQUE(train_l.user_id)>,
 <Feature: lectures.TIME_SINCE_LAST(train_l.timestamp)>,
 <Feature: users.AVG_TIME_BETWEEN(train_l.timestamp)>,
 <Feature: users.COUNT(train_l)>,
 <Feature: users.LAST(train_l.content_id)>,
 <Feature: users.LAST(train_l.row_id)>,
 <Feature: users.LAST(train_l.task_container_id)>,
 <Feature: users.MODE(train_l.content_id)>,
 <Feature: users.MODE(train_

## train_q

In [86]:
# Train
entity = 'train_q'
entities.add(entity)

agg_primitives[entity] = [
                          # 'sum', 
                          'time_since_last', 
                          'avg_time_between', 
                          'all', 
                          'mode', 
                          'num_unique', 
                          # 'min', 
                          'last', 
                          # 'mean', 
                          'percent_true', 
                          # 'max', 
                          # 'std', 
                          'count',
                          # total_previous_m,
                          # total_previous_w,
                          # total_previous_d,
                          # total_previous_12h,
                          # total_previous_6h,
                          # total_previous_3h,
                          # total_previous_2h,
                          # total_previous_1h
                          ]

trans_primitives[entity] = [
                          # 'cum_sum', 
                          # 'diff', 
                          'time_since_previous',
                          # time_since
                          ]

where_primitives[entity] = [
                            # 'sum', 
                            # 'mean', 
                            # 'percent_true', 
                            # 'all', 
                            # 'any'
                            ]
max_depth[entity] = 2


feature_defs[entity] = ft.dfs(entityset=es, target_entity=entity, 
                      # cutoff_time = cutoff_times, 
                      agg_primitives = agg_primitives[entity],
                      trans_primitives = trans_primitives[entity],
                      where_primitives = where_primitives[entity],
                      chunk_size = 1000, #len(cutoff_times), 
                      # cutoff_time_in_index = True,
                      max_depth = max_depth[entity], 
                      features_only = True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.')


This will generate 246 features for train_q.


In [87]:
print(entity, '\n')
feature_defs[entity]

train_q 



[<Feature: user_id>,
 <Feature: content_id>,
 <Feature: task_container_id>,
 <Feature: user_answer>,
 <Feature: answered_correctly>,
 <Feature: prior_question_had_explanation>,
 <Feature: TIME_SINCE_PREVIOUS(timestamp)>,
 <Feature: questions.bundle_id>,
 <Feature: questions.correct_answer>,
 <Feature: questions.part>,
 <Feature: questions.tag_0>,
 <Feature: questions.tag_1>,
 <Feature: questions.tag_10>,
 <Feature: questions.tag_100>,
 <Feature: questions.tag_101>,
 <Feature: questions.tag_102>,
 <Feature: questions.tag_103>,
 <Feature: questions.tag_104>,
 <Feature: questions.tag_105>,
 <Feature: questions.tag_106>,
 <Feature: questions.tag_107>,
 <Feature: questions.tag_108>,
 <Feature: questions.tag_109>,
 <Feature: questions.tag_11>,
 <Feature: questions.tag_110>,
 <Feature: questions.tag_111>,
 <Feature: questions.tag_112>,
 <Feature: questions.tag_113>,
 <Feature: questions.tag_114>,
 <Feature: questions.tag_115>,
 <Feature: questions.tag_116>,
 <Feature: questions.tag_117>,
 <Fe

## users

In [88]:
# Users

entity = 'users'
entities.add(entity)

agg_primitives[entity] = ['sum', 
                          'time_since_last', 
                          'avg_time_between', 
                          'all', 
                          'mode', 
                          'num_unique', 
                          'min', 
                          'last', 
                          'mean', 
                          'percent_true', 
                          'max', 
                          'std', 
                          'count',
                          total_previous_m,
                          total_previous_w,
                          total_previous_d,
                          total_previous_12h,
                          total_previous_6h,
                          total_previous_3h,
                          total_previous_2h,
                          total_previous_1h
                          ]

trans_primitives[entity] = [
                          'cum_sum', 
                          'diff', 
                          'time_since_previous',
                          # time_since
                          ]

where_primitives[entity] = ['sum', 
                          'mean', 
                          'percent_true', 
                          'all', 
                          # 'any'
                          ]
max_depth[entity] = 2


feature_defs[entity] = ft.dfs(entityset=es, target_entity=entity, 
                      # cutoff_time = cutoff_times, 
                      agg_primitives = agg_primitives[entity],
                      trans_primitives = trans_primitives[entity],
                      where_primitives = where_primitives[entity],
                      chunk_size = 1000, #len(cutoff_times), 
                      # cutoff_time_in_index = True,
                      max_depth = 2, 
                      features_only = True)

print(f'This will generate {len(feature_defs[entity])} features for {entity}.')

This will generate 1425 features for users.


In [ ]:
print(entity, '\n')
feature_defs[entity]

# Run Deep Feature Synthesis

Once we're happy with the features that will be generated, we can run deep feature synthesis to make the actual features. We need to change `feature_only` to `False` and then we're good to go.

In [100]:
# Define cutoff times
user_cutoff_time = pd.DataFrame(df_train[['user_id','timestamp']])
user_cutoff_time['time'] = user_cutoff_time['timestamp']
# user_cutoff_time.drop(columns=['timestamp'], inplace=True)

cutoff_time = {
    'lectures': None,
    'questions': None,
    'train_l': None,
    'train_q': None,
    'users': user_cutoff_time
}

In [101]:
user_cutoff_time

,user_id,timestamp,time
37497771,801103753,2019-11-16 23:28:53.275,2019-11-16 23:28:53.275
37497772,801103753,2019-11-16 23:29:14.461,2019-11-16 23:29:14.461
37497773,801103753,2019-11-16 23:29:48.305,2019-11-16 23:29:48.305
37497774,801103753,2019-11-16 23:30:15.114,2019-11-16 23:30:15.114
37497775,801103753,2019-11-16 23:30:36.621,2019-11-16 23:30:36.621
...,...,...,...
37515683,801103753,2020-10-09 23:52:48.820,2020-10-09 23:52:48.820
37515684,801103753,2020-10-09 23:54:00.127,2020-10-09 23:54:00.127
37515685,801103753,2020-10-09 23:55:18.179,2020-10-09 23:55:18.179
37515686,801103753,2020-10-09 23:56:26.788,2020-10-09 23:56:26.788


In [102]:
entities

{'lectures', 'questions', 'train_l', 'train_q', 'users'}

In [103]:
from timeit import default_timer as timer

from featuretools.selection import remove_low_information_features,\
                                   remove_highly_null_features, \
                                   remove_highly_correlated_features

# Dict of feature matrix and feature defs
feature_defs = {}
feature_matrix = {}

# Dict of encoded feature matrix and feature defs
fm_enc = {}
f_enc = {}

#Define save paths
feature_defs_paths = {}
feature_matrix_paths = {}

for entity in entities:
  print('\n', entity)
  start = timer()
  fm, fd = ft.dfs(entityset=es, 
                        target_entity=entity, 
                        cutoff_time = cutoff_time[entity], 
                        agg_primitives = agg_primitives[entity],
                        trans_primitives = trans_primitives[entity],
                        where_primitives = where_primitives[entity],
                        max_depth = max_depth[entity], 
                        features_only = False,
                        verbose = 1, 
                        chunk_size = 1000,  
                        # n_jobs = -1,
                        # cutoff_time_in_index = True
                        )

  # Feature Selection
  # Drop features that have less than 2 unique non-null values
  fm, fd = remove_low_information_features(fm, features=fd)

  # Drop features that have more than 95% null values
  fm, fd = remove_highly_null_features(fm, features=fd, pct_null_threshold=0.95)

  # When two features are determined to be highly correlated, 
  # we remove the more complex of the two.
  fm, fd = remove_highly_correlated_features(fm, features=fd, pct_corr_threshold=0.90)

  feature_matrix[entity] = fm
  feature_defs[entity] = fd
  end = timer()
  print(f'{round(end - start)} seconds elapsed.')

  #Save feature defs
  feature_defs_paths[entity] = pwd + '/data/intermediate/feature_defs_' + entity + '.txt' 
  with open(feature_defs_paths[entity], 'w') as f:
    ft.save_features(fd, f)

  #Save feature matrix
  feature_matrix_paths[entity] = pwd + '/data/intermediate/feature_matrix_' + entity +'.pkl.gzip'
  with open(feature_matrix_paths[entity], 'wb') as f:
    fm.to_pickle(f)

train_l 

Built 50 features
Elapsed: 00:00 | Progress: 100%|██████████
1 seconds elapsed.
lectures 

Built 13 features
Elapsed: 00:00 | Progress: 100%|██████████
1 seconds elapsed.
train_q 

Built 246 features
Elapsed: 00:26 | Progress: 100%|██████████
26 seconds elapsed.
users 

Built 1425 features
Elapsed: 13:42:51 | Progress: 100%|██████████
49371 seconds elapsed.
questions 

Built 214 features
Elapsed: 00:30 | Progress: 100%|██████████
31 seconds elapsed.


We saved these feature definitions as a binary file which will allow us to make the same exact features for another entityset of the same format. This is useful when we have multiple partitions and we want to make the same features for each. Instead of remaking the feature definitions, we pass in the same feature definitions to a call to `calculate_feature_matrix`.

In [ ]:
for fm in feature_matrix.values():
  print(fm.shape)
  print(remove_low_information_features(fm))

In [172]:
feature_matrix['lectures'].head()

,tag,part,type_of,COUNT(train_l),LAST(train_l.row_id),LAST(train_l.user_id),MODE(train_l.user_id),NUM_UNIQUE(train_l.user_id)
lecture_id,,,,,,,,
89,159,5,concept,1.0,37498764.0,801103753.0,801103753.0,1.0
100,70,1,concept,1.0,37511803.0,801103753.0,801103753.0,1.0
185,45,6,concept,0.0,NaN,NaN,NaN,NaN
192,79,5,solving question,1.0,37498938.0,801103753.0,801103753.0,1.0
317,156,5,solving question,1.0,37511792.0,801103753.0,801103753.0,1.0


In [173]:
feature_matrix['questions'].head()

,bundle_id,correct_answer,part,tag_0,tag_1,tag_10,tag_100,tag_101,tag_102,tag_103,tag_104,tag_105,tag_106,tag_107,tag_108,tag_109,tag_11,tag_110,tag_111,tag_112,tag_113,tag_114,tag_115,tag_116,tag_117,tag_118,tag_119,tag_12,tag_120,tag_121,tag_122,tag_123,tag_124,tag_125,tag_126,tag_127,tag_128,tag_129,tag_13,tag_130,...,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_9,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99,ALL(train_q.answered_correctly),ALL(train_q.prior_question_had_explanation),AVG_TIME_BETWEEN(train_q.timestamp),COUNT(train_q),LAST(train_q.answered_correctly),LAST(train_q.prior_question_had_explanation),LAST(train_q.row_id),LAST(train_q.task_container_id),LAST(train_q.user_answer),LAST(train_q.user_id),MODE(train_q.task_container_id),MODE(train_q.user_answer),MODE(train_q.user_id),NUM_UNIQUE(train_q.task_container_id),NUM_UNIQUE(train_q.user_answer),NUM_UNIQUE(train_q.user_id),PERCENT_TRUE(train_q.answered_correctly),PERCENT_TRUE(train_q.prior_question_had_explanation),TIME_SINCE_LAST(train_q.timestamp),ALL(train_q.prior_question_had_explanation WHERE answered_correctly = True),ALL(train_q.prior_question_had_explanation WHERE answered_correctly = False),PERCENT_TRUE(train_q.prior_question_had_explanation WHERE answered_correctly = True),PERCENT_TRUE(train_q.prior_question_had_explanation WHERE answered_correctly = False)
question_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.352757e+07,NaN,NaN,0.0,0.0
1,1,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,NaN,1.0,True,True,37498425.0,603.0,1.0,801103753.0,603.0,1.0,801103753.0,1.0,1.0,1.0,1.0,1.0,4.120068e+07,True,NaN,1.0,0.0
2,2,0,1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,4.997219e+06,3.0,False,True,37508632.0,6033.0,3.0,801103753.0,1286.0,1.0,801103753.0,3.0,3.0,1.0,0.0,1.0,4.120065e+07,NaN,True,0.0,1.0
3,3,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,NaN,1.0,True,True,37508171.0,5857.0,0.0,801103753.0,5857.0,0.0,801103753.0,1.0,1.0,1.0,1.0,1.0,3.725894e+07,True,NaN,1.0,0.0
4,4,3,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,NaN,1.0,True,True,37507228.0,5433.0,3.0,801103753.0,5433.0,3.0,801103753.0,1.0,1.0,1.0,1.0,1.0,4.120060e+07,True,NaN,1.0,0.0


In [174]:
feature_matrix['train_l'].head()

,content_id,task_container_id,TIME_SINCE_PREVIOUS(timestamp),lectures.tag,lectures.part,lectures.type_of,lectures.COUNT(train_l),lectures.LAST(train_l.row_id)
row_id,,,,,,,,
37497812,3153,42,NaN,62,2,intention,1,37497812
37497839,21411,69,57485.685,163,2,solving question,1,37497839
37497859,8976,77,179445.998,62,2,intention,1,37497859
37497883,5752,113,127857.806,6,2,intention,1,37497883
37497914,25401,144,86391.282,128,5,concept,1,37497914


In [175]:
feature_matrix['train_q'].head()

,content_id,task_container_id,user_answer,answered_correctly,prior_question_had_explanation,TIME_SINCE_PREVIOUS(timestamp),questions.bundle_id,questions.correct_answer,questions.part,questions.tag_0,questions.tag_1,questions.tag_10,questions.tag_100,questions.tag_101,questions.tag_102,questions.tag_103,questions.tag_104,questions.tag_105,questions.tag_106,questions.tag_107,questions.tag_108,questions.tag_109,questions.tag_11,questions.tag_110,questions.tag_111,questions.tag_112,questions.tag_113,questions.tag_114,questions.tag_115,questions.tag_116,questions.tag_117,questions.tag_118,questions.tag_119,questions.tag_12,questions.tag_120,questions.tag_121,questions.tag_122,questions.tag_123,questions.tag_124,questions.tag_125,...,questions.tag_75,questions.tag_76,questions.tag_77,questions.tag_79,questions.tag_8,questions.tag_80,questions.tag_81,questions.tag_82,questions.tag_83,questions.tag_84,questions.tag_85,questions.tag_86,questions.tag_87,questions.tag_88,questions.tag_89,questions.tag_9,questions.tag_90,questions.tag_91,questions.tag_92,questions.tag_93,questions.tag_94,questions.tag_95,questions.tag_96,questions.tag_97,questions.tag_98,questions.tag_99,questions.ALL(train_q.answered_correctly),questions.ALL(train_q.prior_question_had_explanation),questions.AVG_TIME_BETWEEN(train_q.timestamp),questions.COUNT(train_q),questions.LAST(train_q.answered_correctly),questions.LAST(train_q.prior_question_had_explanation),questions.LAST(train_q.row_id),questions.LAST(train_q.user_answer),questions.MODE(train_q.task_container_id),questions.MODE(train_q.user_answer),questions.NUM_UNIQUE(train_q.user_answer),questions.PERCENT_TRUE(train_q.answered_correctly),questions.PERCENT_TRUE(train_q.prior_question_had_explanation),questions.TIME_SINCE_LAST(train_q.timestamp)
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37497772,4032,1,3,False,False,NaN,4032,1,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1.383657e+07,3,False,False,37514887,3,1,3,1,0.0,0.333333,2.655716e+07
37497773,5927,2,1,True,False,33.844,5927,1,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,NaN,1,True,False,37497773,1,2,1,1,1.0,0.000000,1.576579e+07
37497774,4754,3,2,True,False,26.809,4754,2,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,NaN,1,True,False,37497774,2,3,2,1,1.0,0.000000,3.424797e+07
37497775,6336,4,1,False,False,21.507,6336,0,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3.941688e+06,2,True,True,37500304,0,4,0,2,0.5,0.500000,2.651906e+07
37497776,9301,5,0,True,False,27.706,9301,0,5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,Fals

In [176]:
feature_matrix['users'].head()

,AVG_TIME_BETWEEN(train_l.timestamp),COUNT(train_l),LAST(train_l.content_id),LAST(train_l.row_id),LAST(train_l.task_container_id),MODE(train_l.content_id),MODE(train_l.task_container_id),NUM_UNIQUE(train_l.content_id),NUM_UNIQUE(train_l.task_container_id),TIME_SINCE_LAST(train_l.timestamp),ALL(train_q.answered_correctly),ALL(train_q.prior_question_had_explanation),AVG_TIME_BETWEEN(train_q.timestamp),COUNT(train_q),LAST(train_q.answered_correctly),LAST(train_q.content_id),LAST(train_q.prior_question_had_explanation),LAST(train_q.row_id),LAST(train_q.task_container_id),LAST(train_q.user_answer),MODE(train_q.content_id),MODE(train_q.task_container_id),MODE(train_q.user_answer),NUM_UNIQUE(train_q.content_id),NUM_UNIQUE(train_q.task_container_id),NUM_UNIQUE(train_q.user_answer),PERCENT_TRUE(train_q.answered_correctly),PERCENT_TRUE(train_q.prior_question_had_explanation),TIME_SINCE_LAST(train_q.timestamp),LAST(train_l.TIME_SINCE_PREVIOUS(timestamp)),LAST(train_l.lectures.part),LAST(train_l.lectures.tag),LAST(train_l.lectures.type_of),MAX(train_l.TIME_SINCE_PREVIOUS(timestamp)),MEAN(train_l.TIME_SINCE_PREVIOUS(timestamp)),MIN(train_l.TIME_SINCE_PREVIOUS(timestamp)),MODE(train_l.lectures.part),MODE(train_l.lectures.tag),MODE(train_l.lectures.type_of),NUM_UNIQUE(train_l.lectures.part),...,PERCENT_TRUE(train_q.questions.tag_95 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_95 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_96),PERCENT_TRUE(train_q.questions.tag_96 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_96 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_97),PERCENT_TRUE(train_q.questions.tag_97 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_97 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_98),PERCENT_TRUE(train_q.questions.tag_98 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_98 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_99),PERCENT_TRUE(train_q.questions.tag_99 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_99 WHERE answered_correctly = False),STD(train_q.TIME_SINCE_PREVIOUS(timestamp)),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp)),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp) WHERE answered_correctly = True),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp) WHERE answered_correctly = False),"TOTAL_PREVIOUS_1_HOUR(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_2_HOURS(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_3_HOURS(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_HALF_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_MONTH(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_QUARTER_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_WEEK(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)",CUM_SUM(AVG_TIME_BETWEEN(train_l.timestamp)),CUM_SUM(AVG_TIME_BETWEEN(train_q.timestamp)),CUM_SUM(COUNT(train_l)),CUM_SUM(COUNT(train_q)),CUM_SUM(NUM_UNIQUE(train_l.content_id)),CUM_SUM(NUM_UNIQUE(train_l.task_container_id)),CUM_SUM(NUM_UNIQUE(train_q.content_id)),CUM_SUM(NUM_UNIQUE(train_q.task_container_id)),CUM_SUM(NUM_UNIQUE(train_q.user_answer)),CUM_SUM(PERCENT_TRUE(train_q.answered_correctly)),CUM_SUM(PERCENT_TRUE(train_q.prior_question_had_explanation)),CUM_SUM(TIME_SINCE_LAST(train_l.timestamp)),CUM_SUM(TIME_SINCE_LAST(train_q.timestamp)),timestamp
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
801103753,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.00000

# One Hot encode the feature matrtixes

In [170]:
# Encode the feature matrix using One-Hot encoding
feature_defs_enc = {}
feature_matrix_enc = {}

for entity in entities:
  fm_enc, f_enc = ft.encode_features(feature_matrix[entity], feature_defs[entity])
  fm_enc = fm_enc.fillna(0)
  fm_enc = remove_low_information_features(fm_enc)
  feature_defs_enc[entity] = f_enc
  feature_matrix_enc[entity] = fm_enc

In [171]:
# Save the encoded feature matrixes

import os

try:
  os.mkdir(pwd + '/data/intermediate/onehotencoded/')
except:
  pass

# define paths and save files
feature_defs_enc_paths = {}
for entity in entities:
  feature_defs_enc_paths[entity] = pwd + '/data/intermediate/onehotencoded/feature_defs_' + entity + '.txt' 
  with open(feature_defs_enc_paths[entity], 'w') as f:
    ft.save_features(feature_defs[entity], f)

feature_matrix_enc_paths = {}
for entity in entities:
  feature_matrix_enc_paths[entity] = pwd + '/data/intermediate/onehotencoded/feature_matrix_' + entity +'.pkl.gzip'
  with open(feature_matrix_enc_paths[entity], 'wb') as f:
    feature_matrix[entity].to_pickle(f)

In [178]:
feature_matrix_enc['lectures']

,tag = 136,tag = 161,tag = 134,tag = 116,tag = 113,tag = 74,tag = 62,tag = 27,tag = 173,tag = 119,tag is unknown,part = 5,part = 6,part = 2,part = 1,part = 7,part = 4,part = 3,type_of = concept,type_of = solving question,type_of = intention,type_of = starter,COUNT(train_l),LAST(train_l.row_id) = 37515614.0,LAST(train_l.row_id) = 37515551.0,LAST(train_l.row_id) = 37515296.0,LAST(train_l.row_id) = 37515232.0,LAST(train_l.row_id) = 37515065.0,LAST(train_l.row_id) = 37515015.0,LAST(train_l.row_id) = 37514927.0,LAST(train_l.row_id) = 37514878.0,LAST(train_l.row_id) = 37514773.0,LAST(train_l.row_id) = 37514645.0,LAST(train_l.row_id) is unknown,LAST(train_l.user_id) = 801103753.0,LAST(train_l.user_id) is unknown,MODE(train_l.user_id) = 801103753.0,MODE(train_l.user_id) is unknown,NUM_UNIQUE(train_l.user_id)
lecture_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
89,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
100,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
185,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,0.0,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,0.0
192,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
317,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32535,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
32570,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,1.0,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,1.0
32604,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,0.0,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,0.0


In [179]:
feature_matrix_enc['questions']

,bundle_id = 11868,bundle_id = 11863,bundle_id = 11689,bundle_id = 11684,bundle_id = 11679,bundle_id = 11580,bundle_id = 11575,bundle_id = 11552,bundle_id = 8159,bundle_id = 8154,bundle_id is unknown,correct_answer = 0,correct_answer = 3,correct_answer = 1,correct_answer = 2,part = 5,part = 2,part = 3,part = 4,part = 6,part = 7,part = 1,tag_0,tag_1,tag_10,tag_100,tag_101,tag_102,tag_103,tag_104,tag_105,tag_106,tag_107,tag_108,tag_109,tag_11,tag_110,tag_111,tag_112,tag_113,...,LAST(train_q.task_container_id) = 9525.0,LAST(train_q.task_container_id) = 9017.0,LAST(train_q.task_container_id) = 9016.0,LAST(train_q.task_container_id) = 9014.0,LAST(train_q.task_container_id) is unknown,LAST(train_q.user_answer) = 0.0,LAST(train_q.user_answer) = 3.0,LAST(train_q.user_answer) = 1.0,LAST(train_q.user_answer) = 2.0,LAST(train_q.user_answer) is unknown,LAST(train_q.user_id) = 801103753.0,LAST(train_q.user_id) is unknown,MODE(train_q.task_container_id) = 6167.0,MODE(train_q.task_container_id) = 6111.0,MODE(train_q.task_container_id) = 6097.0,MODE(train_q.task_container_id) = 6095.0,MODE(train_q.task_container_id) = 6093.0,MODE(train_q.task_container_id) = 6005.0,MODE(train_q.task_container_id) = 6003.0,MODE(train_q.task_container_id) = 6001.0,MODE(train_q.task_container_id) = 4069.0,MODE(train_q.task_container_id) = 3738.0,MODE(train_q.task_container_id) is unknown,MODE(train_q.user_answer) = 0.0,MODE(train_q.user_answer) = 1.0,MODE(train_q.user_answer) = 3.0,MODE(train_q.user_answer) = 2.0,MODE(train_q.user_answer) is unknown,MODE(train_q.user_id) = 801103753.0,MODE(train_q.user_id) is unknown,NUM_UNIQUE(train_q.task_container_id),NUM_UNIQUE(train_q.user_answer),NUM_UNIQUE(train_q.user_id),PERCENT_TRUE(train_q.answered_correctly),PERCENT_TRUE(train_q.prior_question_had_explanation),TIME_SINCE_LAST(train_q.timestamp),ALL(train_q.prior_question_had_explanation WHERE answered_correctly = True),ALL(train_q.prior_question_had_explanation WHERE answered_correctly = False),PERCENT_TRUE(train_q.prior_question_had_explanation WHERE answered_correctly = True),PERCENT_TRUE(train_q.prior_question_had_explanation WHERE answered_correctly = False)
question_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,0.0,0.0,0.0,0.0,0.0,1.352757e+07,0,0,0.0,0.0
1,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,1.0,1.0,1.0,1.0,1.0,4.120068e+07,True,0,1.0,0.0
2,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,3.0,3.0,1.0,0.0,1.0,4.120065e+07,0,True,0.0,1.0
3,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,Fa

In [180]:
feature_matrix_enc['train_l']

,content_id = 30958,content_id = 32736,content_id = 32625,content_id = 32570,content_id = 32535,content_id = 32312,content_id = 32264,content_id = 32175,content_id = 32168,content_id = 32146,content_id is unknown,task_container_id = 9937,task_container_id = 9881,task_container_id = 9684,task_container_id = 9636,task_container_id = 9536,task_container_id = 9514,task_container_id = 9426,task_container_id = 9394,task_container_id = 9289,task_container_id = 9206,task_container_id is unknown,TIME_SINCE_PREVIOUS(timestamp),lectures.tag = 136,lectures.tag = 161,lectures.tag = 113,lectures.tag = 74,lectures.tag = 62,lectures.tag = 119,lectures.tag = 114,lectures.tag = 94,lectures.tag = 82,lectures.tag = 70,lectures.tag is unknown,lectures.part = 5,lectures.part = 2,lectures.part = 1,lectures.part = 6,lectures.part = 4,lectures.part = 3,lectures.part = 7,lectures.type_of = concept,lectures.type_of = solving question,lectures.type_of = intention,lectures.COUNT(train_l),lectures.LAST(train_l.row_id) = 37501736,lectures.LAST(train_l.row_id) = 37515614,lectures.LAST(train_l.row_id) = 37515551,lectures.LAST(train_l.row_id) = 37515296,lectures.LAST(train_l.row_id) = 37515232,lectures.LAST(train_l.row_id) = 37515065,lectures.LAST(train_l.row_id) = 37515015,lectures.LAST(train_l.row_id) = 37514927,lectures.LAST(train_l.row_id) = 37514878,lectures.LAST(train_l.row_id) = 37514773,lectures.LAST(train_l.row_id) is unknown
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37497812,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,0.000,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,1,False,False,False,False,False,False,False,False,False,False,True
37497839,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,57485.685,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,1,False,False,False,False,False,False,False,False,False,False,True
37497859,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,179445.998,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,1,False,False,False,False,False,False,False,False,False,False,True
37497883,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,127857.806,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,1,False,False,False,False,False,False,False,False,False,False,True
37497914,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,86391.282,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,1,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37515065,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,39630.736,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,1,False,False,False,False,False,True,False,False,False,False,False
37515232,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,95418.272,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,Fal

In [181]:
feature_matrix_enc['train_q']

,content_id = 3350,content_id = 3349,content_id = 3348,content_id = 4811,content_id = 1756,content_id = 1755,content_id = 1754,content_id = 853,content_id = 4804,content_id = 4608,content_id is unknown,task_container_id = 9950,task_container_id = 9854,task_container_id = 9689,task_container_id = 9688,task_container_id = 9687,task_container_id = 9635,task_container_id = 9596,task_container_id = 9531,task_container_id = 9525,task_container_id = 9395,task_container_id is unknown,user_answer = 0,user_answer = 3,user_answer = 1,user_answer = 2,answered_correctly,prior_question_had_explanation,TIME_SINCE_PREVIOUS(timestamp),questions.bundle_id = 3348,questions.bundle_id = 7270,questions.bundle_id = 6805,questions.bundle_id = 1754,questions.bundle_id = 3180,questions.bundle_id = 2371,questions.bundle_id = 1913,questions.bundle_id = 10070,questions.bundle_id = 8159,questions.bundle_id = 8134,questions.bundle_id is unknown,...,questions.ALL(train_q.answered_correctly),questions.ALL(train_q.prior_question_had_explanation),questions.AVG_TIME_BETWEEN(train_q.timestamp),questions.COUNT(train_q),questions.LAST(train_q.answered_correctly),questions.LAST(train_q.prior_question_had_explanation),questions.LAST(train_q.row_id) = 37505929,questions.LAST(train_q.row_id) = 37505928,questions.LAST(train_q.row_id) = 37505927,questions.LAST(train_q.row_id) = 37513630,questions.LAST(train_q.row_id) = 37510777,questions.LAST(train_q.row_id) = 37510776,questions.LAST(train_q.row_id) = 37510775,questions.LAST(train_q.row_id) = 37509694,questions.LAST(train_q.row_id) = 37513629,questions.LAST(train_q.row_id) = 37513296,questions.LAST(train_q.row_id) is unknown,questions.LAST(train_q.user_answer) = 0,questions.LAST(train_q.user_answer) = 3,questions.LAST(train_q.user_answer) = 1,questions.LAST(train_q.user_answer) = 2,questions.MODE(train_q.task_container_id) = 1648,questions.MODE(train_q.task_container_id) = 2563,questions.MODE(train_q.task_container_id) = 2754,questions.MODE(train_q.task_container_id) = 1684,questions.MODE(train_q.task_container_id) = 1721,questions.MODE(train_q.task_container_id) = 1627,questions.MODE(train_q.task_container_id) = 1580,questions.MODE(train_q.task_container_id) = 4144,questions.MODE(train_q.task_container_id) = 2947,questions.MODE(train_q.task_container_id) = 2746,questions.MODE(train_q.task_container_id) is unknown,questions.MODE(train_q.user_answer) = 0,questions.MODE(train_q.user_answer) = 1,questions.MODE(train_q.user_answer) = 3,questions.MODE(train_q.user_answer) = 2,questions.NUM_UNIQUE(train_q.user_answer),questions.PERCENT_TRUE(train_q.answered_correctly),questions.PERCENT_TRUE(train_q.prior_question_had_explanation),questions.TIME_SINCE_LAST(train_q.timestamp)
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37497772,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,0.000,False,False,False,False,False,False,False,False,False,False,True,...,False,False,1.383657e+07,3,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,1,0.000000,0.333333,2.655716e+07
37497773,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,33.844,False,False,False,False,False,False,False,False,False,False,True,...,True,False,0.000000e+00,1,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,1,1.000000,0.000000,1.576579e+07
37497774,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,26.809,False,False,False

In [182]:
feature_matrix_enc['users']

,AVG_TIME_BETWEEN(train_l.timestamp),COUNT(train_l),LAST(train_l.content_id) = 24633.0,LAST(train_l.content_id) = 27402.0,LAST(train_l.content_id) = 13909.0,LAST(train_l.content_id) = 21747.0,LAST(train_l.content_id) = 12354.0,LAST(train_l.content_id) = 27400.0,LAST(train_l.content_id) = 15359.0,LAST(train_l.content_id) = 2476.0,LAST(train_l.content_id) = 100.0,LAST(train_l.content_id) = 7385.0,LAST(train_l.content_id) is unknown,LAST(train_l.row_id) = 37505773.0,LAST(train_l.row_id) = 37505193.0,LAST(train_l.row_id) = 37508521.0,LAST(train_l.row_id) = 37507803.0,LAST(train_l.row_id) = 37511136.0,LAST(train_l.row_id) = 37509447.0,LAST(train_l.row_id) = 37506816.0,LAST(train_l.row_id) = 37504889.0,LAST(train_l.row_id) = 37511803.0,LAST(train_l.row_id) = 37504207.0,LAST(train_l.row_id) is unknown,LAST(train_l.task_container_id) = 4779.0,LAST(train_l.task_container_id) = 4508.0,LAST(train_l.task_container_id) = 5991.0,LAST(train_l.task_container_id) = 5741.0,LAST(train_l.task_container_id) = 7311.0,LAST(train_l.task_container_id) = 6356.0,LAST(train_l.task_container_id) = 5273.0,LAST(train_l.task_container_id) = 4319.0,LAST(train_l.task_container_id) = 7722.0,LAST(train_l.task_container_id) = 3941.0,LAST(train_l.task_container_id) is unknown,MODE(train_l.content_id) = 30958.0,MODE(train_l.content_id) = 89.0,MODE(train_l.content_id) = 484.0,MODE(train_l.content_id) = 3153.0,MODE(train_l.content_id) = 1105.0,...,PERCENT_TRUE(train_q.questions.tag_95 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_95 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_96),PERCENT_TRUE(train_q.questions.tag_96 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_96 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_97),PERCENT_TRUE(train_q.questions.tag_97 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_97 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_98),PERCENT_TRUE(train_q.questions.tag_98 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_98 WHERE answered_correctly = False),PERCENT_TRUE(train_q.questions.tag_99),PERCENT_TRUE(train_q.questions.tag_99 WHERE answered_correctly = True),PERCENT_TRUE(train_q.questions.tag_99 WHERE answered_correctly = False),STD(train_q.TIME_SINCE_PREVIOUS(timestamp)),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp)),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp) WHERE answered_correctly = True),SUM(train_q.TIME_SINCE_PREVIOUS(timestamp) WHERE answered_correctly = False),"TOTAL_PREVIOUS_1_HOUR(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_2_HOURS(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_3_HOURS(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_HALF_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_MONTH(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_QUARTER_DAY(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)","TOTAL_PREVIOUS_WEEK(train_q.TIME_SINCE_PREVIOUS(timestamp), timestamp)",CUM_SUM(AVG_TIME_BETWEEN(train_l.timestamp)),CUM_SUM(AVG_TIME_BETWEEN(train_q.timestamp)),CUM_SUM(COUNT(train_l)),CUM_SUM(COUNT(train_q)),CUM_SUM(NUM_UNIQUE(train_l.content_id)),CUM_SUM(NUM_UNIQUE(train_l.task_container_id)),CUM_SUM(NUM_UNIQUE(train_q.content_id)),CUM_SUM(NUM_UNIQUE(train_q.task_container_id)),CUM_SUM(NUM_UNIQUE(train_q.user_answer)),CUM_SUM(PERCENT_TRUE(train_q.answered_correctly)),CUM_SUM(PERCENT_TRUE(train_q.prior_question_had_explanation)),CUM_SUM(TIME_SINCE_LAST(train_l.timestamp)),CUM_SUM(TIME_SINCE_LAST(train_q.timestamp)),timestamp
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
801103753,0.000000,0.0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,F